In [10]:
import pandas as pd
import re

ukb_data = pd.read_csv("data/ukb_synthetic_data_with_noise.csv")
ukb_data.rename(columns=lambda x: re.sub('-0.0', '', x), inplace=True)
ukb_data.rename(columns=lambda x: x.strip(), inplace=True)
ukb_data.rename(columns=lambda x: x.lower(), inplace=True)
ukb_data.drop(['eid', 'accession', 'subject id', 'ega sample id'], axis=1, inplace=True)
pd.DataFrame(ukb_data.columns, columns=['fields'])

ukb_data.head(10)

,sample name,platelet count,ethnic background,body mass index (bmi),weight,diabetes diagnosed by doctor,age high blood pressure diagnosed,triglycerides,age at recruitment,age diabetes diagnosed,...,operative procedures - opcs4,frequency of drinking alcohol,"blood, blood-forming organs and certain immune disorders","endocrine, nutritional and metabolic diseases",mental and behavioural disorders,nervous system disorders,digestive system disorders,respiratory system disorders,circulatory system disorders,covid-19 tests results
0,UKB_FAKE_1,244.9589,African,26.638383,85.6358,Yes,22.18,1.8299,35.45,21.53,...,L46.3 Ligation of visceral branch of abdominal...,Monthly or less,NaN,NaN,NaN,alzheimer's disease,NaN,NaN,NaN,Negative
1,UKB_FAKE_2,215.7736,British,25.622784,64.5722,Yes,33.61,1.4641,31.38,21.01,...,K06.1 Repositioning of transposed great arteries,4 or more times a week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
2,UKB_FAKE_3,312.3504,Asian or Asian British,22.359763,64.4364,Prefer not to answer,NaN,2.0471,45.40,NaN,...,T89.4 Cannulation of lymphatic duct,2 to 3 times a week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
3,UKB_FAKE_4,150.0247,White and Asian,28.278855,74.4885,Prefer not to answer,NaN,1.4921,47.24,NaN,...,T77.2 Wide excision of muscle,2 to 3 times a week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
4,UKB_FAKE_5,215.5871,White and Black African,25.520670,82.3472,Yes,NaN,1.5540,39.16,22.00,...,Z01.4 Tissue of occipital lobe of brain,Never,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
5,UKB_FAKE_6,230.3244,Prefer not to answer,30.943636,81.8213,Prefer not to answer,37.11,-0.2066,46.10,NaN,...,E55.9 Unspecified open extirpation of lesion o...,2 to 3 times a week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
6,UKB_FAKE_7,114.8653,Any other Asian background,29.095139,80.6949,Prefer not to answer,NaN,1.5180,33.67,NaN,...,Z98.6 Short saphenous vein,4 or more times a week,NaN,NaN,NaN,NaN,NaN,acute bronchitis,NaN,Negative
7,UKB_FAKE_8,192.6149,White and Black Caribbean,19.081154,65.3527,No,34.68,1.5295,41.80,NaN,...,C46.3 Penetrating graft to cornea,2 to 3 times a week,NaN,NaN,NaN,multiple sclerosis,NaN,influenza due to certain identified influenza ...,cardiomyopathy,Negative
8,UKB_FAKE_9,172.5093,Black or Black British,28.173368,94.9065,Yes,NaN,1.4005,45.76,19.08,...,C01.8 Other specified excision of eye,Prefer not to answer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
9,UKB_FAKE_10,224.7824,Any other mixed background,15.284914,44.8895,No,NaN,2.3088,42.85,NaN,...,H03.8 Other specified other operations on appe...,4 or more times a week,NaN,NaN,NaN,NaN,NaN,acute bronchitis,NaN,Negative


In [13]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")
# dictionary_path = 'data/frequency_dictionary_with_mondo.txt'
# bigram_path = 'data/frequency_bigramdictionary_with_mondo.txt'

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

disorder_values = ukb_data['blood, blood-forming organs and certain immune disorders'].dropna().unique()
suggestion_list = []
for val in disorder_values:
    # suggestions = sym_spell.lookup(val, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
    suggestions = sym_spell.lookup_compound(val, max_edit_distance=2)
    suggestion_list.append({'value': val, 'suggestion': suggestions[0].term})

pd.DataFrame(suggestion_list)

,value,suggestion
0,iron deficiency anaemia,iron deficiency anaemia
1,agranulocytosis,agranulocytosis
2,ugraaulocytosis,agranulocytosis
3,sarcoidosis,sarcoid sis
4,amranulocytosts,agranulocytosis
5,iron defisiency anaemia,iron deficiency anaemia
6,uarcoidosis,sarcoid sis
7,agravulocytosis,agranulocytosis
8,iron deficijncy anaemia,iron deficiency anaemia
9,sarcjidlsis,sarcoid sis


In [4]:
disorder_mappings = {match['value']: match['suggestion'] for match in suggestion_list}
ukb_data['blood, blood-forming organs and certain immune disorders'] = \
    ukb_data['blood, blood-forming organs and certain immune disorders'].map(disorder_mappings)
pd.DataFrame(ukb_data['blood, blood-forming organs and certain immune disorders'].unique())


,0
0,NaN
1,iron deficiency anaemia
2,agranulocytosis
3,sarcoidosis
4,pron deficiency anaemia
